In [2]:
import pandas as pd
import geopandas as gpd
import numpy as np

from bokeh.plotting import figure, show, save
from bokeh.models import GeoJSONDataSource, ColumnDataSource, HoverTool
from bokeh.io import export_png

import shapely


import sys

sys.path.append("../")

from Code.helper_functions import initial_loading
from Code.loading_data import load_api_data

POSTGRES password?WelkomCorne!


In [3]:
df_zo = pd.read_csv('../Results/20200510 - cluster optimization/hillclimber_best_config20200509-1228.csv')
all_households, rel_poi_df, joined, df_afstandn2 = initial_loading()

Do you want to use addresses instead of clusters?True
What stadsdeel do you want to make as a subsection(optional parameter)?
What is the maximum amount of containers in a cluster that is considered to be useful?8
Where to get db files(local/online)?local
DB relation POIs loaded


..\Code\loading_data.py:389: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['ligtin_bag_pnd_identificatie'] = \
..\Code\loading_data.py:397: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  verblijfsobjecten['bag'] = verblijfsobjecten.loc[:, 'bag']\


distance matrix loaded
API data loaded
Table all households created
API and DB joined
containers per cluster determined
move_rest determined


In [12]:
joined['textiel'].sum()

329

In [22]:
import math
len(str(math.comb(12183,329) * math.comb(11854, 1336) * math.comb(10518, 1875) * math.comb(8643, 1011)))

5959

In [5]:
from Code.helper_functions import add_shortest_distances_to_all_households, analyze_candidate_solution
from Code.loading_data import create_aansluitingen

joined_cluster_distance = joined.set_index('s1_afv_nodes')\
    .join(df_afstandn2.set_index('van_s1_afv_nodes')).reset_index()\
    .rename(columns={'index': 'van_s1_afv_nodes'})


In [13]:
temp = joined_cluster_distance.sort_values(by='afstand')
temp = temp.groupby('naar_s1_afv_nodes').first()
((temp['afstand'] * temp['count']).sum()) / (temp['count'].sum())

98.88088508902544

#### from Code.helper_functions import add_shortest_distances_to_all_households, analyze_candidate_solution
from Code.loading_data import create_aansluitingen

joined_cluster_distance = joined.set_index('s1_afv_nodes')\
    .join(df_afstandn2.set_index('van_s1_afv_nodes')).reset_index()\
    .rename(columns={'index': 'van_s1_afv_nodes'})

good_result = \
    add_shortest_distances_to_all_households(all_households,
                                             joined_cluster_distance,
                                             use_count=True)

aansluitingen = create_aansluitingen(good_result, joined_cluster_distance, use_count=True)

oined_cluster_distance, good_result_rich, aansluitingen,avg_distance, penalties = analyze_candidate_solution(joined, all_households, rel_poi_df, df_afstandn2, clean=True, use_count=True, return_all=True)

## Zuidoost

In [ ]:
df_zo = df_zo.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1)

In [ ]:
to_join = df_zo[['s1_afv_nodes', 'rest', 'plastic', 'papier', 'glas', 'textiel', 'totaal']].set_index('s1_afv_nodes')
joined.s1_afv_nodes = joined.s1_afv_nodes.astype('int')
new = to_join.join(joined.set_index('s1_afv_nodes'), lsuffix='_n')
new.plot(kind='scatter', x='cluster_x', y='cluster_y', c='totaal')

In [ ]:
households_zo = all_households[(all_households['cluster_x'] > 123600) & (all_households['cluster_y'] < 482600)]
households_zo = households_zo.set_index('naar_s1_afv_nodes').join(df_afstandn2[['count', 'naar_s1_afv_nodes']].set_index('naar_s1_afv_nodes'), how='left').drop_duplicates()

In [ ]:
load = gpd.read_file('../data/Inzameling_huisvuil_100220.shp')

street_map = load[load['sdcode'] == 'T']
geosource = GeoJSONDataSource(geojson = street_map.to_json())
street_map_clean = street_map[street_map['aanbiedwij'] == 'Breng uw restafval  naar een container voor restafval.']
geosource2 = GeoJSONDataSource(geojson = street_map_clean.to_json())

TOOLTIPS = [
("index", "$index"),
("(x,y)", "($x, $y)"),
("Rest", "(@rest, @rest_n)"),
("Plastic", "(@plastic, @plastic_n)"),
("Papier", "(@papier, @papier_n)"),
("Glas", "(@glas, @glas_n)"),
("Textiel", "(@textiel, @textiel_n)")
]

TOOLTIPS2 = [
("index", "$index"),
("(x,y)", "($x, $y)"),
("Count", "@count")]

source1 = ColumnDataSource(data=new[['cluster_x', 'cluster_y', 'rest', 'papier', 'plastic', 'textiel', 'glas', 'rest_n', 'papier_n', 'glas_n', 'plastic_n', 'textiel_n']])
source2 = ColumnDataSource(data=households_zo)


p = figure()
r1 = p.patches('xs', 'ys', source=geosource, fill_color='grey', alpha=0.1, line_color=None)
r2 = p.patches('xs', 'ys', source=geosource2, fill_color='grey', alpha=0.3, line_color=None)
r3 = p.circle(x='cluster_x', y='cluster_y', color='red', line_color=None, source=source1, radius=20)
p.add_tools(HoverTool(renderers=[r3], tooltips=TOOLTIPS))
# p.circle(x=new[new['rest'] > 0]['cluster_x'], y=new[new['rest'] > 0]['cluster_y'], fill_color='red', line_color=None, radius=15, alpha=0.5)
# p.circle(x=new[new['rest'] == 0]['cluster_x'], y=new[new['rest'] == 0]['cluster_y'], fill_color='orange', line_color=None, radius=15, alpha=0.5)
r4 = p.circle(x='cluster_x', y='cluster_y', radius=1.5, source=source2)
p.add_tools(HoverTool(renderers=[r4], tooltips=TOOLTIPS2))


show(p)

In [ ]:
# Bokeh interactive representation
def to_bokeh_interactive(df):
    TOOLS="crosshair,pan,wheel_zoom,zoom_in,zoom_out,box_zoom,undo,redo,reset,tap,save,box_select,poly_select,lasso_select,"
    
    # Configuring hover over tooltip
    TOOLTIPS = [
    ("index", "$index"),
    ("(x,y)", "($x, $y)"),
    ("Rest", "@rest"),
    ("Plastic", "@plastic"),
]

    colors = {'Rest':'grey', 'Papier':'red', 'Plastic':'orange', 'Glas':'green', 'Textiel': 'blue', 'Brood': 'purple', None: 'pink'}
    p = figure(tools=TOOLS, tooltips = TOOLTIPS, title="Garbage Containers in Amsterdam", x_range=(110000, 135000), y_range=(475000, 495000), plot_width = 750, plot_height=600)
    
    # To be able to name them
    for fraction in df.fractie.unique():
        if fraction != None:
            df_temp = df[df['fractie'] == fraction]
            p.circle(x = df_temp['x'], y = df_temp['y'], fill_color=colors[fraction], line_width=0, radius=10, name=fraction, legend_label=str(fraction))
    p.patches('xs', 'ys', source=geosource, fill_color='grey', alpha=0.2)
#     p.patches('xs', 'ys', source=geosource2, fill_color='grey', alpha=0.2)
    p.patches('xs', 'ys', source=geosource3, fill_color='grey', alpha=0.2)
    p.legend.location = "bottom_left"
    output_file("garbage_containers.html")
    show(p)
    return p

## Other pieces

In [5]:
def visualise_configuration():
    coordinates_dict = {'T':[123600, 131000, 476500, 482700],
                        'N':[118000, 135000, 486700, 495000]}
    filename = input("Please provide the requested filename")
    area = input("Provide Code of neighborhood (T/N/F/C)")
    if area == 'C':
        area = ['M', 'A', 'K', 'E']
    df_zo = pd.read_csv(filename)
#     df_zo = df_zo.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1)
    
    all_households, rel_poi_df, joined, df_afstandn2 = initial_loading()
    
    to_join = df_zo[['s1_afv_nodes', 'rest', 'plastic', 'papier', 'glas', 'textiel', 'totaal']].set_index('s1_afv_nodes')
    joined.s1_afv_nodes = joined.s1_afv_nodes.astype('int')
    new = to_join.join(joined.set_index('s1_afv_nodes'), lsuffix='_n')
    
#     households_zo = all_households[(all_households['cluster_x'] > coordinates_dict[area][0]) & (all_households['cluster_x'] < coordinates_dict[area][1]) & (all_households['cluster_y'] > coordinates_dict[area][2]) & (all_households['cluster_y'] < coordinates_dict[area][3])]
    households_zo = all_households[all_households['in_neigborhood']]
    households_zo = households_zo.set_index('naar_s1_afv_nodes').join(df_afstandn2[['count', 'naar_s1_afv_nodes']].set_index('naar_s1_afv_nodes'), how='left').drop_duplicates()

    load = gpd.read_file('../data/Inzameling_huisvuil_100220.shp')
    street_map = load[load['sdcode'].isin(list(area))]
    geosource = GeoJSONDataSource(geojson = street_map.to_json())
    street_map_clean = street_map[street_map['aanbiedwij'] == 'Breng uw restafval  naar een container voor restafval.']
    geosource2 = GeoJSONDataSource(geojson = street_map_clean.to_json())
    

    TOOLTIPS = [
    ("index", "$index"),
    ("(x,y)", "($x, $y)"),
    ("Rest", "(@rest, @rest_n)"),
    ("Plastic", "(@plastic, @plastic_n)"),
    ("Papier", "(@papier, @papier_n)"),
    ("Glas", "(@glas, @glas_n)"),
    ("Textiel", "(@textiel, @textiel_n)")
    ]

    TOOLTIPS2 = [
    ("index", "$index"),
    ("(x,y)", "($x, $y)"),
    ("Count", "@count")]

    source1 = ColumnDataSource(data=new[['cluster_x', 'cluster_y', 'rest', 'papier', 'plastic', 'textiel', 'glas', 'rest_n', 'papier_n', 'glas_n', 'plastic_n', 'textiel_n']])
    source2 = ColumnDataSource(data=households_zo[households_zo['count'] > 0])
    source3 = ColumnDataSource(data=households_zo[households_zo['count'].isna()])


    p = figure(match_aspect=True)
    r1 = p.patches('xs', 'ys', source=geosource, fill_color='grey', alpha=0.1, line_color=None)
    r2 = p.patches('xs', 'ys', source=geosource2, fill_color='grey', alpha=0.3, line_color=None)
    r3 = p.circle(x='cluster_x', y='cluster_y', color='red', line_color=None, source=source1, radius=20)
    p.add_tools(HoverTool(renderers=[r3], tooltips=TOOLTIPS))
    r4 = p.circle(x='cluster_x', y='cluster_y', radius=1.5, source=source2)
    p.add_tools(HoverTool(renderers=[r4], tooltips=TOOLTIPS2))
    r5 = p.circle(x='cluster_x', y='cluster_y', radius=1.5, source=source3, alpha = 0.3, fill_color='green', line_color=None)
    p.add_tools(HoverTool(renderers=[r5], tooltips=TOOLTIPS2))

    show(p)
    return p

In [12]:
kaartje = visualise_configuration()

Please provide the requested filenamehillclimber_best_config20200605-0003.csv
Provide Code of neighborhood (T/N/F/C)C
Do you want to use addresses instead of clusters?True
What stadsdeel do you want to make as a subsection(optional parameter)?
What is the maximum amount of containers in a cluster that is considered to be useful?8
Where to get db files(local/online)?local
DB relation POIs loaded
distance matrix loaded
API data loaded
Table all households created
API and DB joined
containers per cluster determined
move_rest determined


In [13]:
save(kaartje, 'centrum.html')
export_png(kaartje, filename='centrum.png')

'C:\\Users\\corne\\OneDrive\\Documenten\\ISDS\\Master_Thesis\\Amsterdam_carbage_thesis\\Notebooks\\centrum.png'

In [ ]:
def calculate_penalties(good_result, aansluitingen, use_count=False,
                        w_rest=0.35, w_plas=0.25, w_papi=0.2, w_glas=0.15,
                        w_text=0.05, return_all=False):
    """
    Calculate the amount of penalties based on described policies.

    This function calculates all the penalties associated with the candidate
    solution. It does this by calculating the number of times all constraints
    are violated and applies the weighing that is associated with all these
    violations. D_... contain the maximum allowed walking distance to a
    container of the specified fraction. P_... show the maximum amount of
    households connected to a container of the fraction specified.

    Input:
    dataframe good_result containing per adress or adress poi the distance
    to the nearest container for all fractions.
    dataframe connections containing for all clusters the amount of containers
    per fraction, the amount of people using these containers and percentage
    of occupancy compared to the norm

    Output:
    The sum of all different penalties as a single float
    """

    fractions = {'rest': [100, 100, 0.35], 'plastic':[150, 200, 0.25], 'papier':[150, 200, 0.2], 'glas':[150, 200, 0.15], 'textiel':[200, 750, 0.05]}
    MAX_PERC = 100
    NORMAL = 250
    penalties = []
    for k,v in fractions.items():
        dist_pen = good_result[good_result[f'{k}_afstand'] > v[0]]
        conn_pen = aansluitingen[aansluitingen[f'{k}_perc'] > MAX_PERC]
        if not use_count:
            print('a')
            penalties.append((dist_pen[f'{k}_afstand'] - v[0]).sum() / good_result.shape[0] * v[2])
            penalties.append((conn_pen[f'poi_{k}'] - (conn_pen[k] * v[1])).sum() / good_result['count'].sum() * v[2] * NORMAL)
        else:
            print('b')
            penalties.append(((dist_pen[f'{k}_afstand'] -v[0])* dist_pen['count']).sum()/good_result['count'].sum() * v[2])
            penalties.append((conn_pen[f'poi_{k}'] - (conn_pen[k] * v[1])).sum() / good_result['count'].sum() * v[2] * NORMAL)

    if return_all:
        return penalties
    else:
        return sum(penalties)

In [ ]:
calculate_penalties(good_result, aansluitingen, use_count=True, return_all=True)

In [ ]:
df = pd.read_csv('../Data/households_per_cluster.csv')
df1 = df[df['ligtin_bag_pnd_identificatie'].str.len() == 16]
df2 = df[df['ligtin_bag_pnd_identificatie'].str.len() != 16]
df2['ligtin_bag_pnd_identificatie'] = df2['ligtin_bag_pnd_identificatie'].str.split('|').map(lambda x: x[0])
df = df1.append([df2])
df['bag'] = df['ligtin_bag_pnd_identificatie'].astype('int64')
df = df.drop(['ligtin_bag_pnd_identificatie'], axis=1)

In [ ]:
df_afstandn = df_afstandn2.drop(['count'], axis=1)

verblijfsobjecten = rel_poi_df[rel_poi_df['type'] != 'afval_cluster']
verblijfsobjecten['bag'] = verblijfsobjecten['bag'].astype('int64')
temp = verblijfsobjecten.set_index('bag').join(df.set_index('bag'), how='left')
temp['count'] = temp['aantal_woonfunctie']

joined = temp.set_index('s1_afv_nodes')\
        .join(df_afstandn.set_index('naar_s1_afv_nodes'), how='outer')
joined = joined.reset_index()[['van_s1_afv_nodes', 'index', 'afstand', 'count']].\
    rename(columns={'index': 'naar_s1_afv_nodes'}).sort_values(by='afstand').\
    reset_index().drop(['index'], axis=1).dropna()

KeyError: 'count'